# Reactive covering

## Introduction

EXPLAIN

In [ ]:
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cf
import math
from docplex.mp.model import Model

import random
import json

def showSolution(data,solution):
    
    # print the booking strategy
    print("Bookings:")
    for booking in solution['Bookings']:
        print(booking)

    # show the passes
    minVisiDuration = data['minVisiDuration']
    AOIs = data['AOIs']
    nAOIs = len(AOIs)
    accesses = data['Accesses']
    satPasses = data['Passes']
    nPasses = len(satPasses)
    
    ncols = 3
    nrows = math.ceil(nPasses / ncols)
    proj = ccrs.PlateCarree()
    fig, axs = plt.subplots(nrows=nrows,ncols=ncols, subplot_kw={'projection': proj},figsize=(30,80))
    axs = axs.flatten()

    for i in range(nPasses):
        satPass = satPasses[i]
        ax = axs[i]
        ax.set_extent([-15, 25, 35, 60])
        ax.stock_img()
        ax.add_feature(cf.COASTLINE, lw=1)
        ax.add_feature(cf.BORDERS)
        #plt.gcf().set_size_inches(15, 15)    
        for aoi in  AOIs:
            ax.plot([aoi['lon']], [aoi['lat']], 'kx')
        ax.set_title("Pass #{} [{},{}]".format(i,satPass['startDate'],satPass['endDate']))
    
        booking = None
        for bk in solution['Bookings']:
            if bk['passId'] == i:
                booking = bk            
                break
        
        for accessId in satPass['accessIds']:
            access = accesses[accessId]
            aoi = AOIs[access['aoiId']]            
            ax.plot([aoi['lon']], [aoi['lat']], 'rx')
            # show the reservations
            if (booking != None) and (booking['bookingStart'] <= access['end'] - minVisiDuration) and (access['start'] + minVisiDuration <= booking['bookingEnd']):
                ax.plot([aoi['lon']], [aoi['lat']], 'bo')

    plt.show()

## Stage 1

Explicar como hemos calculado el coste y como hemos definido la quality y meter un poco de paja rollo intro

In [ ]:
# Import the data
dataFile = open('data/data12.json')  
data = json.load(dataFile)

# Load data
obsRequests = data['ObservationRequests']
fixedPassCost = data['fixedPassCost']
minVisiDuration = data['minVisiDuration']
aois = data['AOIs']
accesses = data['Accesses']
referenceDate = data['referenceDate']
passCostPerTimeUnit = data['passCostPerTimeUnit']
goals = data['Goals']
satPasses = data['Passes']

# Lengths of lists
nObsRequests = len(obsRequests)
nObsRequestsToCover = nObsRequests
nPasses = len(satPasses)


# Very basic solver
# We start considering all the AOIs uncovered
covered = [False for i in range(nObsRequests)]
remainingSatPassesIds = list(range(nPasses))
random.shuffle(remainingSatPassesIds)
selectedPassIds = []
for i in remainingSatPassesIds:
    # Try to select the ith path
    satPass = satPasses[i]
    keepPass = False
    for accessId in satPass['accessIds']:
        access = accesses[accessId]
        for j in range(nObsRequests):
            obsRequest = obsRequests[j]
            if not covered[j] and accessId in obsRequest['accessIds']:
                covered[j] = True
                keepPass = True
                nObsRequestsToCover -= 1
                
    if keepPass:
        selectedPassIds.append(i)
            
    if nObsRequestsToCover == 0:
        break

# Computation of the total cost
timeUses = [satPasses[id]['end'] - satPasses[id]['start'] for id in selectedPassIds]
totalCost = len(selectedPassIds)*data['fixedPassCost'] + sum(timeUses)*data['passCostPerTimeUnit']

# Computation of the quality of the covering
quality = 0
for passId in selectedPassIds:
    timeUse = satPasses[passId]['end'] - satPasses[passId]['start']
    for accessId in satPasses[passId]['accessIds']:
        for obsRequest in obsRequests:
            if accessId in obsRequest['accessIds']:
                quality += goals[obsRequest['goalId']]['priority']*timeUse

# Presentation of the results
print("IDs of the {0} selected passes: {1}".format(len(selectedPassIds), selectedPassIds))
print("Total cost of the covering: {0}".format(totalCost))
print("Quality of the covering: {0}".format(quality))

# Export the solution to json
bookings = []
for i in selectedPassIds:
    satPass = satPasses[i]
    bookings.append({"passId": i, "passStart": satPass['start'], "passEnd": satPass['end'], "bookingStart": satPass['start'], "bookingEnd": satPass['end']})
jsonString = json.dumps({"Bookings": bookings})
jsonFile = open("mySolution.json", "w")
jsonFile.write(jsonString)
jsonFile.close()

In [ ]:
# import the solution
solutionFile = open('mySolution.json')  
solution = json.load(solutionFile)

showSolution(data,solution)

## Stage 2

EXPLAIN

In [13]:
# Import the data
dataFile = open('data/data12.json')  
data = json.load(dataFile)

# Load data
obsRequests = data['ObservationRequests']
fixedPassCost = data['fixedPassCost']
minVisiDuration = data['minVisiDuration']
aois = data['AOIs']
accesses = data['Accesses']
referenceDate = data['referenceDate']
passCostPerTimeUnit = data['passCostPerTimeUnit']
goals = data['Goals']
satPasses = data['Passes']

# Lengths of lists
nObsRequests = len(obsRequests)
nObsRequestsToCover = nObsRequests
nPasses = len(satPasses)

# Parameters
maxBudget = 3735

# OPTIMISATION MODEL
# Model creation
model = Model("reactiveCovering")

# Variables
selectedPass = model.binary_var_list(range(nPasses), name='selectedPass')
tStart = model.continuous_var_list(range(nPasses), name='tStart')
tEnd = model.continuous_var_list(range(nPasses), name='tEnd')

# Objectif: 
# 1) Maximisation of the quality of the covering
def qualityCompForPass(passId):
    quality = 0
    for accessId in satPasses[passId]['accessIds']:
        for obsRequest in obsRequests:
            if accessId in obsRequest['accessIds']:
                quality += goals[obsRequest['goalId']]['priority']*(satPasses[passId]['end'] - satPasses[passId]['start'])*selectedPass[passId]
    return quality

model.maximize(model.sum(qualityCompForPass(passId) for passId in range(nPasses)))

# Contraints:
# 1) Cost below maximum budget
model.add_constraint(model.sum(selectedPass)*data['fixedPassCost'] + model.sum((satPasses[id]['end'] - satPasses[id]['start'])*selectedPass[id] for id in range(nPasses)) <= maxBudget)

# Solve:
# Computation time limit
model.set_time_limit(300)

model.print_information()

# Call of the solver
solution = model.solve(log_output=True)

# Getting the solution
selectedPassIds = []
if solution is None:
    raise Exception("No solution")
else:
    for i in range(nPasses):
        if selectedPass[i].solution_value == 1:
            selectedPassIds.append(i)

# Computation of the total cost
timeUses = [satPasses[id]['end'] - satPasses[id]['start'] for id in selectedPassIds]
totalCost = len(selectedPassIds)*data['fixedPassCost'] + sum(timeUses)*data['passCostPerTimeUnit']

# Computation of the quality of the covering (NO HACE FALTA)
quality = 0
for passId in selectedPassIds:
    timeUse = satPasses[passId]['end'] - satPasses[passId]['start']
    for accessId in satPasses[passId]['accessIds']:
        for obsRequest in obsRequests:
            if accessId in obsRequest['accessIds']:
                quality += goals[obsRequest['goalId']]['priority']*timeUse

# Presentation of the results
print("IDs of the {0} selected passes: {1}".format(len(selectedPassIds), selectedPassIds))
print("Total cost of the covering: {0}".format(totalCost))
print("Quality of the covering: {0}".format(solution.objective_value))

# export the solution to json
bookings = []
for i in selectedPassIds:
    satPass = satPasses[i]
    bookings.append({"passId": i, "passStart": satPass['start'], "passEnd": satPass['end'], "bookingStart": satPass['start'], "bookingEnd": satPass['end']})
jsonString = json.dumps({"Bookings": bookings})
jsonFile = open("mySolution.json", "w")
jsonFile.write(jsonString)
jsonFile.close()


Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               60
Found incumbent of value 0.000000 after 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 2 rows and 142 columns.
MIP Presolve added 11 rows and 11 columns.
Reduced MIP has 10 rows, 76 columns, and 94 nonzeros.
Reduced MIP has 65 binaries, 11 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.25 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 11 rows and 11 columns.
MIP Presolve added 11 rows and 11 columns.
Reduced MIP has 10 rows, 76 columns, and 94 nonzeros.
Reduced MIP has 65 binaries, 11 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.16 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to

In [ ]:
# import the solution
solutionFile = open('mySolution.json')  
solution = json.load(solutionFile)

showSolution(data,solution)

## Stage 3

EXPLAIN

In [27]:
# Import the data
dataFile = open('data/data12.json')  
data = json.load(dataFile)

# Load data
obsRequests = data['ObservationRequests']
fixedPassCost = data['fixedPassCost']
minVisiDuration = data['minVisiDuration']
aois = data['AOIs']
accesses = data['Accesses']
referenceDate = data['referenceDate']
passCostPerTimeUnit = data['passCostPerTimeUnit']
goals = data['Goals']
satPasses = data['Passes']

# Lengths of lists
nObsRequests = len(obsRequests)
nObsRequestsToCover = nObsRequests
nPasses = len(satPasses)

# Parameters
maxBudget = 3735 # Modify if required

# OPTIMISATION MODEL
# Model creation
model = Model("reactiveCovering")

# Variables
selectedPass = model.binary_var_list(range(nPasses), name='selectedPass')

######################################################
######################################################
tStart = model.continuous_var_list(range(nPasses), name='tStart')
tEnd = model.continuous_var_list(range(nPasses), name='tEnd')
######################################################
######################################################

# Objectif
# Computation of the quality of the covering
# AQUI HAY ALGUN FALLO
#model.maximize(model.sum(goals[obsRequest['goalId']]['priority']*(satPasses[passId]['end'] - satPasses[passId]['start'])*selectedPass[passId]*int(accessId in obsRequest['accessIds']) 
#            for passId in range(nPasses) for obsRequest in obsRequests for accessId in satPasses[passId]['accessIds']))

# Revisar
def qualityCompForPass(passId):
    quality = 0
    for accessId in satPasses[passId]['accessIds']:
        for obsRequest in obsRequests:
            if accessId in obsRequest['accessIds']:                 #############################
                quality += goals[obsRequest['goalId']]['priority']*(tEnd[passId] - tStart[passId])*selectedPass[passId]
    return quality                                                  #############################

model.maximize(model.sum(qualityCompForPass(passId) for passId in range(nPasses)))

# Contraints:
# 1) Cost below maximum budget
#model.add_constraint(model.sum(alpha[i, b]*T[i] for i in Stations) <= Gamma)
#totalCost = len(selectedPassIds)*data['fixedPassCost'] + sum([satPasses[id]['end'] - satPasses[id]['start'] for id in selectedPassIds])*data['passCostPerTimeUnit']
                                                                                #####################
model.add_constraint(model.sum(selectedPass)*data['fixedPassCost'] + model.sum((tEnd[id] - tStart[id])*selectedPass[id] for id in range(nPasses)) <= maxBudget)
                                                                                #####################

######################################################
###################################################### AQUI FALLA, creo que en los <= y >=
# 2) tStart constraint
model.add_constraint(tStart[id] >= satPasses[id]['start'] for id in range(nPasses))
# 3) tEnd constraint
model.add_constraint(tEnd[id] <= satPasses[id]['end'] for id in range(nPasses))
# 4) (tEnd-tStart) >= 30s constraint
model.add_constraint(((tEnd[id] - tStart[id]) >= 30) for id in range(nPasses))

# model.le_constraint ???
# model.vector_compare_le ???
######################################################
######################################################


# Solve
# Time limitation
model.set_time_limit(60)

#model.print_information()

# Call of the solver
solution = model.solve(log_output=True)

solution.display()

# Getting the solution
selectedPassIds = []
if solution is None:
    raise Exception("No solution")
else:
    for i in range(nPasses):
        if selectedPass[i].solution_value == 1:
            selectedPassIds.append(i)

# Computation of the total cost
            #####################
timeUses = [tEnd[id] - tStart[id] for id in selectedPassIds]
            #####################
totalCost = len(selectedPassIds)*data['fixedPassCost'] + sum(timeUses)*data['passCostPerTimeUnit']

# Computation of the quality of the covering (NO HACE FALTA)
quality = 0
for passId in selectedPassIds:
    timeUse = satPasses[passId]['end'] - satPasses[passId]['start']
    for accessId in satPasses[passId]['accessIds']:
        for obsRequest in obsRequests:
            if accessId in obsRequest['accessIds']:
                quality += goals[obsRequest['goalId']]['priority']*timeUse

# Presentation of the results
print("IDs of the {0} selected passes: {1}".format(len(selectedPassIds), selectedPassIds))
print("Total cost of the covering: {0}".format(totalCost))
print("Quality of the covering: {0}".format(solution.objective_value))

# export the solution to json
bookings = []
for i in selectedPassIds:
    satPass = satPasses[i]
    bookings.append({"passId": i, "passStart": satPass['start'], "passEnd": satPass['end'], "bookingStart": satPass['start'], "bookingEnd": satPass['end']})
jsonString = json.dumps({"Bookings": bookings})
jsonFile = open("mySolution.json", "w")
jsonFile.write(jsonString)
jsonFile.close()

DOcplexException: Expecting constraint, got: <generator object <genexpr> at 0x0000022D24C75F50> with type: <class 'generator'>

In [ ]:
# import the solution
solutionFile = open('mySolution.json')  
solution = json.load(solutionFile)

showSolution(data,solution)